In [1]:
dataset_name = "cifar10"
model_type = "cnn" 
nump_epochs = 40
pth = "./trajectories"

In [ ]:
!pip install ipywidgets

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F



img_size =  4
ch_size = 3
# Define the ConvNet architecture
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(ch_size, 8, kernel_size=3, padding=1)  # 1 input channel, 8 output channels
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3, padding=1) # 8 input channels, 16 output channels
        self.conv3 = nn.Conv2d(16, 32, kernel_size=3, padding=1) # 8 input channels, 16 output channels
        self.fc1 = nn.Linear(32 * img_size * img_size, 120)   # Fully connected layer with 120 neurons
        self.fc2 = nn.Linear(120, 10)           # Output layer with 10 neurons (10 classes)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.relu(F.max_pool2d(self.conv3(x), 2))
        x = x.view(-1, 32 * img_size * img_size)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)



True


In [4]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

# Instantiate early stopping
early_stopping = EarlyStopping(patience=3, min_delta=0.01)

In [5]:
if dataset_name == "mnist":
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
    train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)
else:
    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    transform2 = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)                              
    test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform2)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)


# Define the training loop
def train(epoch, model, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data.cuda())
        loss = F.nll_loss(output, target.cuda())
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# Define the test loop
def test(epoch, model, optimizer):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data.cuda())
            test_loss += F.nll_loss(output, target.cuda(), reduction='sum').item()  # Sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max log-probability
            correct += pred.eq(target.cuda().view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n')
    return test_loss


Files already downloaded and verified


In [ ]:
import os


# Initialize the model and optimizer
model = ConvNet()
model=model.cuda()
optimizer = optim.SGD(model.parameters(), lr= 0.001, momentum=0.9, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,  'min',factor=0.5, patience=2)


    
    
def save_model(epoch, model):
    # Save the model after each epoch
    save_path = f'{pth}/models_{model_type}_{dataset_name}/model_epoch_{epoch}.pt'
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    torch.save(model.state_dict(), save_path)
    print(f'Model saved to {save_path}')
    
# Train and test the model with early stopping
save_model(0, model)
for epoch in range(1, nump_epochs + 1):
    train(epoch, model, optimizer)
    val_loss = test(epoch, model, optimizer)
    scheduler.step(val_loss)
    early_stopping(val_loss)    
    save_model(epoch, model)
    if early_stopping.early_stop:
        print("Early stopping triggered")
        break
    



In [ ]:
from torchsummary import summary

img_dim  =4 *8
summary(model, input_size=(3, img_dim, img_dim))  # Input size should match the input dimensions of your model
